In [11]:
sc

In [5]:
import csv

In [160]:
spark

In [161]:
sqlContext.dropTempTable

In [12]:
#data =  sc.textFile('coalesce_joined_weather_collisions_sorted_fixed.csv')
df = spark.read.load('coalesce_joined_weather_collisions_sorted_fixed.csv', format='csv', header=True, inferSchema=False)
df.cache()


#df.registerTempTable('df')
sqlContext.registerDataFrameAsTable(df, 'df')


from pyspark.sql.types import IntegerType

inj_kil = ["NUMBER_OF_CYCLIST_INJURED",
"NUMBER_OF_CYCLIST_KILLED",
"NUMBER_OF_MOTORIST_INJURED",
"NUMBER_OF_MOTORIST_KILLED",
"NUMBER_OF_PEDESTRIANS_INJURED",
"NUMBER_OF_PEDESTRIANS_KILLED",
"NUMBER_OF_PERSONS_INJURED",
"NUMBER_OF_PERSONS_KILLED"]

for x in inj_kil:
    df = df.withColumn(x, df[x].cast(IntegerType()))




In [ ]:
if __name__=='__main__':
    sc = SparkContext()
    spark = SparkSession(sc)
    sqlContext = SQLContext(sc)    
    df = spark.read.load('hdfs:///user/dzeng01/project/joined_weather_collisions_sorted_fixed.csv', format='csv', header=True, inferSchema=False)
temp = spark.sql(query)    

In [7]:
df.take(2)

[Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'70 DRIVE', DATETIME=u'2016-06-20 21:06:00', LATITUDE=None, LOCATION=None, LONGITUDE=None, NUMBER_OF_CYCLIST_INJURED=0, NUMBER_OF_CYCLIST_KILLED=0, NUMBER_OF_MOTORIST_INJURED=0, NUMBER_OF_MOTORIST_KILLED=0, NUMBER_OF_PEDESTRIANS_INJURED=0, NUMBER_OF_PEDESTRIANS_KILLED=0, NUMBER_OF_PERSONS_INJURED=0, NUMBER_OF_PERSONS_KILLED=0, OFF_STREET_NAME=None, ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=u'3465402', VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=u'11375', _c28=u'58671', Conditions=u'Clear', DateUTC=u'2016-06-21 01:51:00', DatetimeEDT=u'2016-06-20 21:51:00', Dew_PointF=u'63.0', Events=None, Freezing=u'0', Gust_SpeedM

In [3]:
df.dtypes

[('BOROUGH', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_1', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_2', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_3', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_4', 'string'),
 ('CONTRIBUTING_FACTOR_VEHICLE_5', 'string'),
 ('CROSS_STREET_NAME', 'string'),
 ('DATETIME', 'string'),
 ('LATITUDE', 'string'),
 ('LOCATION', 'string'),
 ('LONGITUDE', 'string'),
 ('NUMBER_OF_CYCLIST_INJURED', 'int'),
 ('NUMBER_OF_CYCLIST_KILLED', 'int'),
 ('NUMBER_OF_MOTORIST_INJURED', 'int'),
 ('NUMBER_OF_MOTORIST_KILLED', 'int'),
 ('NUMBER_OF_PEDESTRIANS_INJURED', 'int'),
 ('NUMBER_OF_PEDESTRIANS_KILLED', 'int'),
 ('NUMBER_OF_PERSONS_INJURED', 'int'),
 ('NUMBER_OF_PERSONS_KILLED', 'int'),
 ('OFF_STREET_NAME', 'string'),
 ('ON_STREET_NAME', 'string'),
 ('UNIQUE_KEY', 'string'),
 ('VEHICLE_TYPE_CODE_1', 'string'),
 ('VEHICLE_TYPE_CODE_2', 'string'),
 ('VEHICLE_TYPE_CODE_3', 'string'),
 ('VEHICLE_TYPE_CODE_4', 'string'),
 ('VEHICLE_TYPE_CODE_5', 'string'),
 ('ZIP_CODE', 'string')

In [41]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
def filter_general(x):
    if x > 65:
        return float(x)
    else:
        return float(0)
    
gfg = udf(filter_general, FloatType())
newcol = df.withColumn("Dew_PointF", gfg("Dew_PointF"))
newcol.take(5)


[Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'70 DRIVE', DATETIME=u'2016-06-20 21:06:00', LATITUDE=None, LOCATION=None, LONGITUDE=None, NUMBER_OF_CYCLIST_INJURED=u'0', NUMBER_OF_CYCLIST_KILLED=u'0', NUMBER_OF_MOTORIST_INJURED=u'0', NUMBER_OF_MOTORIST_KILLED=u'0', NUMBER_OF_PEDESTRIANS_INJURED=u'0', NUMBER_OF_PEDESTRIANS_KILLED=u'0', NUMBER_OF_PERSONS_INJURED=u'0', NUMBER_OF_PERSONS_KILLED=u'0', OFF_STREET_NAME=None, ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=u'3465402', VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=u'11375', _c28=u'58671', Conditions=u'Clear', DateUTC=u'2016-06-21 01:51:00', DatetimeEDT=u'2016-06-20 21:51:00', Dew_PointF=63.0, Events=None, Freez

DataFrame[BOROUGH: string, CONTRIBUTING_FACTOR_VEHICLE_1: string, CONTRIBUTING_FACTOR_VEHICLE_2: string, CONTRIBUTING_FACTOR_VEHICLE_3: string, CONTRIBUTING_FACTOR_VEHICLE_4: string, CONTRIBUTING_FACTOR_VEHICLE_5: string, CROSS_STREET_NAME: string, DATETIME: string, LATITUDE: string, LOCATION: string, LONGITUDE: string, NUMBER_OF_CYCLIST_INJURED: string, NUMBER_OF_CYCLIST_KILLED: string, NUMBER_OF_MOTORIST_INJURED: string, NUMBER_OF_MOTORIST_KILLED: string, NUMBER_OF_PEDESTRIANS_INJURED: string, NUMBER_OF_PEDESTRIANS_KILLED: string, NUMBER_OF_PERSONS_INJURED: string, NUMBER_OF_PERSONS_KILLED: string, OFF_STREET_NAME: string, ON_STREET_NAME: string, UNIQUE_KEY: string, VEHICLE_TYPE_CODE_1: string, VEHICLE_TYPE_CODE_2: string, VEHICLE_TYPE_CODE_3: string, VEHICLE_TYPE_CODE_4: string, VEHICLE_TYPE_CODE_5: string, ZIP_CODE: string, _c28: string, Conditions: string, DateUTC: string, DatetimeEDT: string, Dew_PointF: string, Events: string, Freezing: string, Gust_SpeedMPH: string, Humidity: s

In [10]:
# sqlContext.sql('select * from df where Rain=1')
from pyspark.sql.functions import col
from calendar import monthrange
import json


#date = ""

# Yearly statistics


def months(x):
    month = {1:"January",
             2:"Feburary",
             3:"March",
             4:"April",
             5:"May",
             6:"June",
             7:"July",
             8:"August",
             9:"September",
             10:"October",
             11:"November",
             12:"December"}
    return str(month[x])



for yr in range(16,18):
    
    #### This should be code that will read in each csv for each year and register as df.
    stats = {}
    stats["0_YEAR"] = int(str(20)+str(yr))   
    
    
    ## Overall Statistics
    stats['Overall'] = {}
    m_arr = [] # Overall Collision Data
    mi_arr = [] # Overall Collision Injury Data
    md_arr = [] # Overall Collision Death Data
    
    """PREPROCESSING"""
    # Overall Collision Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}"'.format(date)
        sq = sqlContext.sql(query)
        m_arr.append((x, sq.count())) 
        sq.unpersist()
        
    total = sum([x[1] for x in m_arr])
     
    # Overall Collision Injury Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
        df_injs = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
        mi_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_injs.unpersist()
        
    injury_total = sum([x[1] for x in mi_arr])
    injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in mi_arr])
      
    # Overall Collision Death Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
        df_deths = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
        md_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_deths.unpersist()
        
    death_total = sum([x[1] for x in md_arr])
    death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in md_arr])

    
    
    
    
    
    
    # Statistics Deriving
    stats['Overall']['Total Number of Persons Injured in Year'] = injury_persons_total
    stats['Overall']['Total Number of Persons Injured Per Month'] = [str(
                                                                (months(x+1), sum(map(lambda a: a[0]*a[1], mi_arr[x][2]['Cyclist'])) + 
                                                                              sum(map(lambda a: a[0]*a[1], mi_arr[x][2]['Motorist'])) + 
                                                                              sum(map(lambda a: a[0]*a[1], mi_arr[x][2]['Pedestrians'])))
                                                                ) for x in range(0,12)]
    stats['Overall']['Total Number of Persons Killed in Year'] = death_persons_total
    stats['Overall']['Total Number of Persons Killed Per Month'] = [str(
                                                                (months(x+1), sum(map(lambda a: a[0]*a[1], md_arr[x][2]['Cyclist'])) + 
                                                                              sum(map(lambda a: a[0]*a[1], md_arr[x][2]['Motorist'])) + 
                                                                              sum(map(lambda a: a[0]*a[1], md_arr[x][2]['Pedestrians'])))
                                                                ) for x in range(0,12)]
    stats['Overall']['Total Number of Collisions in Year'] = total
    stats['Overall']['Total Number of Collisions w/ Injury in Year'] = injury_total
    stats['Overall']['Total Number of Collisions w/ Death in Year'] = death_total
    stats['Overall']['Average Collisions Per Month'] = total // 12
    stats['Overall']['Average Collisions Per Day'] = total // 365
    stats['Overall']['Monthly Adjusted Average Collision Weight (%)'] = [str(
                                                                (months(x+1), round((float(m_arr[x][1])/float(total)*100.0),2))
                                                                ) for x in range(0,12)]
    stats['Overall']['Collisions Per Month'] = [str(
                                                (months(x+1), m_arr[x][1])
                                                ) for x in range(0,12)]
    stats['Overall']['Average Collisions Per Day (Monthly)'] = [str(
                                                                (months(x+1), m_arr[x][1]/monthrange(int(str(20)+str(yr)), x+1)[1])
                                                                ) for x in range(0,12)]
    stats['Overall']['Injury Rate (Year %)'] = float(injury_total)/float(total) * 100.00
    stats['Overall']['Death Rate (Year %)'] = float(death_total)/float(total) * 100.00
    stats['Overall']['Number of Collisions w/ Injuries Per Month'] = [str(
                                                                    (months(x+1), mi_arr[x][1])
                                                                    ) for x in range(0,12)]
    stats['Overall']['Number of Collisions w/ Deaths Per Month'] = [str(
                                                                    (months(x+1), md_arr[x][1])
                                                                    ) for x in range(0,12)]
    stats['Overall']['Injury Collision Rate Per Month (Month %)'] = [str(
                                                        (months(x+1), round((float(mi_arr[x][1])/float(m_arr[x][1]) * 100.00),2))
                                                        ) for x in range(0,12) if m_arr[x][1] != 0]
    stats['Overall']['Death Collision Rate Per Month (Month %)'] = [str(
                                                        (months(x+1), round((float(md_arr[x][1])/float(m_arr[x][1]) * 100.00),2))
                                                        ) for x in range(0,12) if m_arr[x][1] != 0]
    
    
    
    

    
    # Rain Statistics
    stats['Rain'] = {}
    r_arr = [] # Rain Collision Data
    ri_arr = [] # Rain Collision Injury Data
    rd_arr = [] # Rain Collision Death Data

    """PREPROCESSING"""
    # Rain Collision Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}"'.format(date)
        sq = sqlContext.sql(query)
        r_arr.append((x, sq.count()))
        sq.unpersist()
    rain_total = sum([x[1] for x in r_arr])
    
    # Rain Collision Injury Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
        df_injs = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
        ri_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_injs.unpersist()
    rain_injury_total = sum([x[1] for x in ri_arr])
    rain_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in ri_arr])
    
    # Rain Collision Death Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Rain="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
        df_deths = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
        rd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_deths.unpersist()
        
    rain_death_total = sum([x[1] for x in rd_arr])
    rain_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in rd_arr])

    
    
    
    # Statistics Deriving
    stats['Rain']['Total Number of Persons Injured in Collision w/ Rain'] = rain_injury_persons_total
    stats['Rain']['Total Number of Persons Killed in Collision w/ Rain'] = rain_death_persons_total
    stats['Rain']['Total Number of Persons Injured in Collision w/ Rain Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], ri_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], ri_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], ri_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['Rain']['Total Number of Persons Killed in Collision w/ Rain Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], rd_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], rd_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], rd_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['Rain']['Total Number of Collisions w/ Injury & Rain in Year'] = rain_injury_total
    stats['Rain']['Total Number of Collisions w/ Death & Rain in Year'] = rain_death_total
    stats['Rain']['Injury Rate w/ Rain (Year %)'] = float(rain_injury_total)/float(rain_total) * 100.00
    stats['Rain']['Death Rate w/ Rain (Year %)'] = float(rain_death_total)/float(rain_total) * 100.00
    stats['Rain']['Total Number of Rain Collisions in Year'] = rain_total
    stats['Rain']['Percent of Collisions in Rain Over Year (%)'] = (float(rain_total) / float(total)) * 100.00
    stats['Rain']['Collisions in Rain'] = [str(
                                        (months(x+1), r_arr[x][1])
                                        ) for x in range(0,12)]
    stats['Rain']['Average Collisions in Rain Per Month'] = rain_total//12
    stats['Rain']['Number of Collisions w/ Injuries and Rain Per Month'] = [str(
                                                                            (months(x+1), ri_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Rain']['Number of Collisions w/ Deaths and Rain Per Month'] = [str(
                                                                            (months(x+1), rd_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Rain']['Percent of Collisions in Each Month That Are Rain Collisions (Rain %)'] = [str(
                                                            (months(x+1), round((float(r_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                            ) for x in range(0,12) if m_arr[x][1] != 0]
    stats['Rain']['Injury Rate Per Month w/ Rain (Month %)'] = [str(
                                                            (months(x+1), round((float(ri_arr[x][1])/float(r_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if r_arr[x][1] != 0]
    stats['Rain']['Death Rate Per Month w/ Rain (Month %)'] = [str(
                                                            (months(x+1), round((float(rd_arr[x][1])/float(r_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if r_arr[x][1] != 0]
    
    
    
    
    # Snow Statistics
    stats['Snow'] = {}
    s_arr = [] # Snow Collision Data
    si_arr = [] # Snow Collision Injury Data
    sd_arr = [] # Snow Collision Death Data

    """PREPROCESSING"""
    # Snow Collision Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Snow="1" and DATETIME like "{0}"'.format(date)
        sq = sqlContext.sql(query)
        s_arr.append((x, sq.count()))
        sq.unpersist()
    snow_total = sum([x[1] for x in s_arr])
    
    # Snow Collision Injury Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Snow="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
        df_injs = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
        si_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_injs.unpersist()
    snow_injury_total = sum([x[1] for x in si_arr])
    snow_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in si_arr])
    
    # Snow Collision Death Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Snow="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
        df_deths = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
        sd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_deths.unpersist()
        
    snow_death_total = sum([x[1] for x in sd_arr])
    snow_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in sd_arr])

    
    
    
    # Statistics Deriving
    stats['Snow']['Total Number of Persons Injured in Collision w/ Snow'] = snow_injury_persons_total
    stats['Snow']['Total Number of Persons Killed in Collision w/ Snow'] = snow_death_persons_total
    stats['Snow']['Total Number of Persons Injured in Collision w/ Snow Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], si_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], si_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], si_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['Snow']['Total Number of Persons Killed in Collision w/ Snow Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], sd_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], sd_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], sd_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['Snow']['Total Number of Collisions w/ Injury & Snow in Year'] = snow_injury_total
    stats['Snow']['Total Number of Collisions w/ Death & Snow in Year'] = snow_death_total
    stats['Snow']['Injury Rate w/ Snow (Year %)'] = float(snow_injury_total)/float(snow_total) * 100.00
    stats['Snow']['Death Rate w/ Snow (Year %)'] = float(snow_death_total)/float(snow_total) * 100.00
    stats['Snow']['Total Number of Snow Collisions in Year'] = snow_total
    stats['Snow']['Percent of Collisions in Snow Over Year (%)'] = (float(snow_total) / float(total)) * 100.00
    stats['Snow']['Collisions in Snow'] = [str(
                                        (months(x+1), s_arr[x][1])
                                        ) for x in range(0,12)]
    stats['Snow']['Average Collisions in Snow Per Month'] = snow_total//12
    stats['Snow']['Number of Collisions w/ Injuries and Snow Per Month'] = [str(
                                                                            (months(x+1), si_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Snow']['Number of Collisions w/ Deaths and Snow Per Month'] = [str(
                                                                            (months(x+1), sd_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Snow']['Percent of Collisions in Each Month That Are Snow Collisions (Snow %)'] = [str(
                                                            (months(x+1), round((float(s_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                            ) for x in range(0,12) if m_arr[x][1] != 0]
    stats['Snow']['Injury Rate Per Month w/ Snow (Month %)'] = [str(
                                                            (months(x+1), round((float(si_arr[x][1])/float(s_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if s_arr[x][1] != 0]
    stats['Snow']['Death Rate Per Month w/ Snow (Month %)'] = [str(
                                                            (months(x+1), round((float(sd_arr[x][1])/float(s_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if s_arr[x][1] != 0]
    
    
    
    
    
    
    
    
    # Freezing Temperatures Statistics
    stats['Freezing'] = {}
    f_arr = [] # Freezing Collision Data
    fi_arr = [] # Freezing Collision Injury Data
    fd_arr = [] # Freezing Collision Death Data

    """PREPROCESSING"""
    # Freezing Collision Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Freezing="1" and DATETIME like "{0}"'.format(date)
        sq = sqlContext.sql(query)
        f_arr.append((x, sq.count()))
        sq.unpersist()
    freezing_total = sum([x[1] for x in f_arr])
    
    # Freezing Collision Injury Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Freezing="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
        df_injs = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
        fi_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_injs.unpersist()
    freezing_injury_total = sum([x[1] for x in fi_arr])
    freezing_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in fi_arr])
    
    # Freezing Collision Death Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Freezing="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
        df_deths = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
        fd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_deths.unpersist()
        
    freezing_death_total = sum([x[1] for x in fd_arr])
    freezing_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in fd_arr])

    
    
    
    # Statistics Deriving
    stats['Freezing']['Total Number of Persons Injured in Collision w/ Freezing'] = freezing_injury_persons_total
    stats['Freezing']['Total Number of Persons Killed in Collision w/ Freezing'] = freezing_death_persons_total
    stats['Freezing']['Total Number of Persons Injured in Collision w/ Freezing Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], fi_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], fi_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], fi_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['Freezing']['Total Number of Persons Killed in Collision w/ Freezing Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], fd_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], fd_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], fd_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['Freezing']['Total Number of Collisions w/ Injury & Freezing in Year'] = freezing_injury_total
    stats['Freezing']['Total Number of Collisions w/ Death & Freezing in Year'] = freezing_death_total
    stats['Freezing']['Injury Rate w/ Freezing (Year %)'] = float(freezing_injury_total)/float(freezing_total) * 100.00
    stats['Freezing']['Death Rate w/ Freezing (Year %)'] = float(freezing_death_total)/float(freezing_total) * 100.00
    stats['Freezing']['Total Number of Freezing Collisions in Year'] = freezing_total
    stats['Freezing']['Percent of Collisions in Freezing Over Year (%)'] = (float(freezing_total) / float(total)) * 100.00
    stats['Freezing']['Collisions in Freezing'] = [str(
                                        (months(x+1), f_arr[x][1])
                                        ) for x in range(0,12)]
    stats['Freezing']['Average Collisions in Freezing Per Month'] = freezing_total//12
    stats['Freezing']['Number of Collisions w/ Injuries and Freezing Per Month'] = [str(
                                                                            (months(x+1), fi_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Freezing']['Number of Collisions w/ Deaths and Freezing Per Month'] = [str(
                                                                            (months(x+1), fd_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Freezing']['Percent of Collisions in Each Month That Are Freezing Collisions (Freezing %)'] = [str(
                                                            (months(x+1), round((float(f_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                            ) for x in range(0,12) if m_arr[x][1] != 0]
    stats['Freezing']['Injury Rate Per Month w/ Freezing (Month %)'] = [str(
                                                            (months(x+1), round((float(fi_arr[x][1])/float(f_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if f_arr[x][1] != 0]
    stats['Freezing']['Death Rate Per Month w/ Freezing (Month %)'] = [str(
                                                            (months(x+1), round((float(fd_arr[x][1])/float(f_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if f_arr[x][1] != 0]
    
    

    
    
    # Low Visibility Statistics
    stats['LowVisibility'] = {}
    lv_arr = [] # Low Visibility Collision Data
    lvi_arr = [] # Low Visibility Collision Injury Data
    lvd_arr = [] # Low Visibility Collision Death Data

    """PREPROCESSING"""
    # Low Visibility Collision Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where LowVisibility="1" and DATETIME like "{0}"'.format(date)
        sq = sqlContext.sql(query)
        lv_arr.append((x, sq.count()))
        sq.unpersist()
    lowvisibility_total = sum([x[1] for x in lv_arr])
    
    # Low Visibility Collision Injury Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where LowVisibility="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
        df_injs = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
        lvi_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_injs.unpersist()
    lowvisibility_injury_total = sum([x[1] for x in lvi_arr])
    lowvisibility_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in lvi_arr])
    
    # Low Visibility Collision Death Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where LowVisibility="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
        df_deths = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
        lvd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_deths.unpersist()
        
    lowvisibility_death_total = sum([x[1] for x in lvd_arr])
    lowvisibility_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in lvd_arr])

    
    
    
    # Statistics Deriving
    stats['LowVisibility']['Total Number of Persons Injured in Collision w/ Low Visibility'] = lowvisibility_injury_persons_total
    stats['LowVisibility']['Total Number of Persons Killed in Collision w/ Low Visibility'] = lowvisibility_death_persons_total
    stats['LowVisibility']['Total Number of Persons Injured in Collision w/ Low Visibility Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], lvi_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], lvi_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], lvi_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['LowVisibility']['Total Number of Persons Killed in Collision w/ Low Visibility Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], lvd_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], lvd_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], lvd_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['LowVisibility']['Total Number of Collisions w/ Injury & Low Visibility in Year'] = lowvisibility_injury_total
    stats['LowVisibility']['Total Number of Collisions w/ Death & Low Visibility in Year'] = lowvisibility_death_total
    stats['LowVisibility']['Injury Rate w/ Low Visibility (Year %)'] = float(lowvisibility_injury_total)/float(lowvisibility_total) * 100.00
    stats['LowVisibility']['Death Rate w/ Low Visibility (Year %)'] = float(lowvisibility_death_total)/float(lowvisibility_total) * 100.00
    stats['LowVisibility']['Total Number of Low Visibility Collisions in Year'] = lowvisibility_total
    stats['LowVisibility']['Percent of Collisions in Low Visibility Over Year (%)'] = (float(lowvisibility_total) / float(total)) * 100.00
    stats['LowVisibility']['Collisions in Low Visibility'] = [str(
                                        (months(x+1), lv_arr[x][1])
                                        ) for x in range(0,12)]
    stats['LowVisibility']['Average Collisions in Low Visibility Per Month'] = lowvisibility_total//12
    stats['LowVisibility']['Number of Collisions w/ Injuries and Low Visibility Per Month'] = [str(
                                                                            (months(x+1), lvi_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['LowVisibility']['Number of Collisions w/ Deaths and Low Visibility Per Month'] = [str(
                                                                            (months(x+1), lvd_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['LowVisibility']['Percent of Collisions in Each Month That Are Low Visibility Collisions (LowVisibility %)'] = [str(
                                                            (months(x+1), round((float(lv_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                            ) for x in range(0,12) if m_arr[x][1] != 0]
    stats['LowVisibility']['Injury Rate Per Month w/ Low Visibility (Month %)'] = [str(
                                                            (months(x+1), round((float(lvi_arr[x][1])/float(lv_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if lv_arr[x][1] != 0]
    stats['LowVisibility']['Death Rate Per Month w/ Low Visibility (Month %)'] = [str(
                                                            (months(x+1), round((float(lvd_arr[x][1])/float(lv_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if lv_arr[x][1] != 0]    
    

    
    

    # Windy Statistics
    stats['Windy'] = {}
    w_arr = [] # Windy Collision Data
    wi_arr = [] # Windy Collision Injury Data
    wd_arr = [] # Windy Collision Death Data

    """PREPROCESSING"""
    # Windy Collision Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Windy="1" and DATETIME like "{0}"'.format(date)
        sq = sqlContext.sql(query)
        w_arr.append((x, sq.count()))
        sq.unpersist()
    windy_total = sum([x[1] for x in w_arr])
    
    # Windy Collision Injury Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Windy="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
        df_injs = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
        wi_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_injs.unpersist()
    windy_injury_total = sum([x[1] for x in wi_arr])
    windy_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in wi_arr])
    
    # Windy Collision Death Data Query
    for x in range(1,13):
        date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
        query = 'select * from df where Windy="1" and DATETIME like "{0}" and ( \
                                                                NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
        df_deths = sqlContext.sql(query)
        cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
        motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
        pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
        wd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
        df_deths.unpersist()
        
    windy_death_total = sum([x[1] for x in wd_arr])
    windy_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in wd_arr])

    
    
    
    # Statistics Deriving
    stats['Windy']['Total Number of Persons Injured in Collision w/ Windy'] = windy_injury_persons_total
    stats['Windy']['Total Number of Persons Killed in Collision w/ Windy'] = windy_death_persons_total
    stats['Windy']['Total Number of Persons Injured in Collision w/ Windy Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], wi_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], wi_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], wi_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['Windy']['Total Number of Persons Killed in Collision w/ Windy Per Month'] = [str(
                                                                        (months(x+1), sum(map(lambda a: a[0]*a[1], wd_arr[x][2]['Cyclist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], wd_arr[x][2]['Motorist'])) + 
                                                                                      sum(map(lambda a: a[0]*a[1], wd_arr[x][2]['Pedestrians'])))
                                                                        ) for x in range(0,12)]    
    stats['Windy']['Total Number of Collisions w/ Injury & Windy in Year'] = windy_injury_total
    stats['Windy']['Total Number of Collisions w/ Death & Windy in Year'] = windy_death_total
    stats['Windy']['Injury Rate w/ Windy (Year %)'] = float(windy_injury_total)/float(windy_total) * 100.00
    stats['Windy']['Death Rate w/ Windy (Year %)'] = float(windy_death_total)/float(windy_total) * 100.00
    stats['Windy']['Total Number of Windy Collisions in Year'] = windy_total
    stats['Windy']['Percent of Collisions in Windy Over Year (%)'] = (float(windy_total) / float(total)) * 100.00
    stats['Windy']['Collisions in Windy'] = [str(
                                        (months(x+1), w_arr[x][1])
                                        ) for x in range(0,12)]
    stats['Windy']['Average Collisions in Windy Per Month'] = windy_total//12
    stats['Windy']['Number of Collisions w/ Injuries and Windy Per Month'] = [str(
                                                                            (months(x+1), wi_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Windy']['Number of Collisions w/ Deaths and Windy Per Month'] = [str(
                                                                            (months(x+1), wd_arr[x][1])
                                                                            ) for x in range(0,12)]
    stats['Windy']['Percent of Collisions in Each Month That Are Windy Collisions (Windy %)'] = [str(
                                                            (months(x+1), round((float(w_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                            ) for x in range(0,12) if m_arr[x][1] != 0]
    stats['Windy']['Injury Rate Per Month w/ Windy (Month %)'] = [str(
                                                            (months(x+1), round((float(wi_arr[x][1])/float(w_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if w_arr[x][1] != 0]
    stats['Windy']['Death Rate Per Month w/ Windy (Month %)'] = [str(
                                                            (months(x+1), round((float(wd_arr[x][1])/float(w_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if w_arr[x][1] != 0]
    
    
    
    
    
    
    
    
    
    json_results = json.dumps(stats, indent=4, sort_keys=True)
    print json_results
    path = "hdfs:///user/dzeng01/project/statistical_results_20{0}".format(yr)
    #sc.parallelize([json_results]).saveAsTextFile(path)
    #print stats


# print arr

# Monthly Statistics

# for x in [date]:
    
#     a = sqlContext.sql('select * from df where Rain=1')
#     a.coalesce(1).write.csv('rain', header=True)

#     a = sqlContext.sql('select * from df where Snow=1')
#     a.coalesce(1).write.csv('snow', header=True)

#     a = sqlContext.sql('select * from df where Freezing=1')
#     a.coalesce(1).write.csv('freezing', header=True)

#     a = sqlContext.sql('select * from df where LowVisibility=1')
#     a.coalesce(1).write.csv('foggy', header=True)

#     a = sqlContext.sql('select * from df where Windy=1')
#     a.coalesce(1).write.csv('windy', header=True)


# df = df.filter(df.LATITUDE != '0')

# from pyspark.sql.functions import col


# a = df.where(col("LOCATION").isNotNull())
# a.coalesce(1).write.csv('geodata', header=True)

{
    "0_YEAR": 2016, 
    "Overall": {
        "Average Collisions Per Day": 295, 
        "Average Collisions Per Day (Monthly)": [
            "('January', 0)", 
            "('Feburary', 0)", 
            "('March', 0)", 
            "('April', 0)", 
            "('May', 0)", 
            "('June', 0)", 
            "('July', 339)", 
            "('August', 634)", 
            "('September', 649)", 
            "('October', 634)", 
            "('November', 641)", 
            "('December', 616)"
        ], 
        "Average Collisions Per Month": 8974, 
        "Collisions Per Month": [
            "('January', 0)", 
            "('Feburary', 0)", 
            "('March', 0)", 
            "('April', 0)", 
            "('May', 0)", 
            "('June', 1)", 
            "('July', 10516)", 
            "('August', 19672)", 
            "('September', 19491)", 
            "('October', 19659)", 
            "('November', 19235)", 
            "('December', 19118)"
        ], 
     

In [186]:
print(float(217417) / float(35408))

6.1403355174


In [205]:
        query = 'select sum(NUMBER_OF_CYCLIST_INJURED) from df'
        a = sqlContext.sql(query)

In [274]:
df.filter(col('DATETIME').like("2016-08%")).groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).show()

+--------------------------+-----+
|NUMBER_OF_MOTORIST_INJURED|count|
+--------------------------+-----+
|                         0|17037|
|                         2| 1546|
|                         4|  472|
|                         6|  199|
|                         3|  156|
|                         1|   82|
|                         8|   72|
|                        12|   31|
|                        10|   26|
|                         9|   17|
|                         5|   14|
|                         7|    7|
|                        14|    3|
|                        28|    1|
|                        22|    1|
|                        20|    1|
|                        16|    1|
|                        23|    1|
|                        19|    1|
|                        15|    1|
+--------------------------+-----+
only showing top 20 rows



In [248]:
from pyspark.sql.functions import col
print df.filter(col('DATETIME').like("2017-03%")).groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).rdd.map(lambda x: (x['NUMBER_OF_CYCLIST_INJURED'], x['count'])).collect()
print df.filter(col('DATETIME').like("2017-03%")).groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).rdd.map(lambda x: (x['NUMBER_OF_MOTORIST_INJURED'], x['count'])).collect()
print df.filter(col('DATETIME').like("2017-03%")).groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).rdd.map(lambda x: (x['NUMBER_OF_PEDESTRIANS_INJURED'], x['count'])).collect()
print df.filter(col('DATETIME').like("2017-03%")).groupBy("NUMBER_OF_PERSONS_INJURED").count().orderBy('count', ascending=False).rdd.map(lambda x: (x['NUMBER_OF_PERSONS_INJURED'], x['count'])).collect()

[(0, 18879), (1, 227), (2, 3)]
[(0, 16968), (1, 1461), (2, 449), (3, 139), (4, 57), (5, 16), (6, 14), (7, 3), (13, 1), (9, 1)]
[(0, 18190), (1, 885), (2, 30), (3, 4)]
[(0, 15839), (1, 2547), (2, 484), (3, 147), (4, 56), (6, 16), (5, 15), (7, 3), (13, 1), (9, 1)]


In [237]:
test

[(0, 18847), (1, 361), (2, 25), (3, 2)]

In [233]:
df.filter(col('DATETIME').like("2016-10%"))

DataFrame[BOROUGH: string, CONTRIBUTING_FACTOR_VEHICLE_1: string, CONTRIBUTING_FACTOR_VEHICLE_2: string, CONTRIBUTING_FACTOR_VEHICLE_3: string, CONTRIBUTING_FACTOR_VEHICLE_4: string, CONTRIBUTING_FACTOR_VEHICLE_5: string, CROSS_STREET_NAME: string, DATETIME: string, LATITUDE: string, LOCATION: string, LONGITUDE: string, NUMBER_OF_CYCLIST_INJURED: int, NUMBER_OF_CYCLIST_KILLED: int, NUMBER_OF_MOTORIST_INJURED: int, NUMBER_OF_MOTORIST_KILLED: int, NUMBER_OF_PEDESTRIANS_INJURED: int, NUMBER_OF_PEDESTRIANS_KILLED: int, NUMBER_OF_PERSONS_INJURED: int, NUMBER_OF_PERSONS_KILLED: int, OFF_STREET_NAME: string, ON_STREET_NAME: string, UNIQUE_KEY: string, VEHICLE_TYPE_CODE_1: string, VEHICLE_TYPE_CODE_2: string, VEHICLE_TYPE_CODE_3: string, VEHICLE_TYPE_CODE_4: string, VEHICLE_TYPE_CODE_5: string, ZIP_CODE: string, _c28: string, Conditions: string, DateUTC: string, DatetimeEDT: string, Dew_PointF: string, Events: string, Freezing: string, Gust_SpeedMPH: string, Humidity: string, LowVisibility: st

In [277]:
map(lambda x: (int(x[0]), x[1]), df.filter(col('DATETIME').like("2016-08%")).select("NUMBER_OF_MOTORIST_INJURED").groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())

[(0, 17037),
 (2, 1546),
 (4, 472),
 (6, 199),
 (3, 156),
 (1, 82),
 (8, 72),
 (12, 31),
 (10, 26),
 (9, 17),
 (5, 14),
 (7, 7),
 (14, 3),
 (28, 1),
 (22, 1),
 (16, 1),
 (20, 1),
 (19, 1),
 (15, 1),
 (23, 1),
 (32, 1),
 (18, 1),
 (36, 1)]

In [243]:
df.select("LOCATION").groupBy("LOCATION").count().orderBy('count', ascending=False).rdd.flatMap(lambda x: x).collect()

[None,
 63518,
 u'(40.7197624, -73.9447576)',
 70,
 u'(40.608757, -74.038086)',
 60,
 u'(40.79522, -73.921524)',
 56,
 u'(40.6916827, -73.9993335)',
 47,
 u'(40.6960346, -73.9845292)',
 45,
 u'(40.8047, -73.91243)',
 43,
 u'(40.820189, -73.8906752)',
 37,
 u'(40.820305, -73.89083)',
 36,
 u'(40.79537, -73.921974)',
 36,
 u'(40.7044959, -73.817418)',
 36,
 u'(40.8451897, -73.9111894)',
 36,
 u'(40.7014547, -73.9896203)',
 34,
 u'(40.602806, -74.062874)',
 34,
 u'(40.8360929, -73.8703818)',
 33,
 u'(40.7606005, -73.9643142)',
 33,
 u'(40.7606, -73.96434)',
 33,
 u'(40.798256, -73.82744)',
 33,
 u'(40.861862, -73.91282)',
 32,
 u'(40.696033, -73.98453)',
 32,
 u'(40.704422, -73.99491)',
 31,
 u'(40.604313, -74.05208)',
 31,
 u'(40.7460781, -73.9749359)',
 31,
 u'(40.805435, -73.9208602)',
 31,
 u'(40.756096, -73.74655)',
 30,
 u'(40.608364, -74.038666)',
 30,
 u'(40.5861184, -73.9360334)',
 30,
 u'(40.688922, -73.9991472)',
 30,
 u'(40.6915381, -73.99911)',
 29,
 u'(40.7615508, -73.966584

In [ ]:
df.select("CONTRIBUTING_FACTOR_VEHICLE_2").distinct().rdd.flatMap(lambda x: x).collect()

In [ ]:
df.select("CONTRIBUTING_FACTOR_VEHICLE_1").distinct().rdd.flatMap(lambda x: x).collect()

In [263]:
        query = 'select * from df where( \
                                                                (NUMBER_OF_CYCLIST_INJURED=0 and \
                                                                NUMBER_OF_MOTORIST_INJURED=0 and \
                                                                NUMBER_OF_PEDESTRIANS_INJURED=0) and \
                                                                NUMBER_OF_PERSONS_INJURED!=0)'
        sqlContext.sql(query).count() 

14

In [271]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
from calendar import monthrange
import json




def months(x):
    month = {1:"January",
             2:"Feburary",
             3:"March",
             4:"April",
             5:"May",
             6:"June",
             7:"July",
             8:"August",
             9:"September",
             10:"October",
             11:"November",
             12:"December"}
    return str(month[x])


if __name__ == '__main__':
    sc = SparkContext()
    spark = SparkSession(sc)
    sqlContext = SQLContext(sc)


    for yr in range(12,18):
        path = "hdfs:///user/dzeng01/project/partitioned_joins_by_year/joins_in_20{0}.csv".format(yr)
        df = spark.read.load(path, format='csv', header=True, inferSchema=False)
        df.cache()
        sqlContext.registerDataFrameAsTable(df, 'df')
        
        inj_kil = ["NUMBER_OF_CYCLIST_INJURED",
        "NUMBER_OF_CYCLIST_KILLED",
        "NUMBER_OF_MOTORIST_INJURED",
        "NUMBER_OF_MOTORIST_KILLED",
        "NUMBER_OF_PEDESTRIANS_INJURED",
        "NUMBER_OF_PEDESTRIANS_KILLED",
        "NUMBER_OF_PERSONS_INJURED",
        "NUMBER_OF_PERSONS_KILLED"]

        for x in inj_kil:
            df = df.withColumn(x, df[x].cast(IntegerType()))




        #### This should be code that will read in each csv for each year and register as df.
        stats = {}
        stats["0_YEAR"] = int(str(20)+str(yr))   


        ## Overall Statistics
        stats['Overall'] = {}
        m_arr = [] # Overall Collision Data
        mi_arr = [] # Overall Collision Injury Data
        md_arr = [] # Overall Collision Death Data

        """PREPROCESSING"""
        # Overall Collision Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where DATETIME like "{0}"'.format(date)
            sq = sqlContext.sql(query)
            m_arr.append((x, sq.count())) 
            sq.unpersist()

        total = sum([x[1] for x in m_arr])

        # Overall Collision Injury Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                    NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
            df_injs = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
            mi_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_injs.unpersist()

        injury_total = sum([x[1] for x in mi_arr])
        injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in mi_arr])

        # Overall Collision Death Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                    NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
            df_deths = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
            md_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_deths.unpersist()

        death_total = sum([x[1] for x in md_arr])
        death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in md_arr])







        # Statistics Deriving
        stats['Overall']['Total Number of Persons Injured in Year'] = injury_persons_total
        stats['Overall']['Total Number of Persons Injured Per Month'] = [str(
                                                                    (months(x+1), sum(map(lambda a: a[0]*a[1], mi_arr[x][2]['Cyclist'])) + 
                                                                                  sum(map(lambda a: a[0]*a[1], mi_arr[x][2]['Motorist'])) + 
                                                                                  sum(map(lambda a: a[0]*a[1], mi_arr[x][2]['Pedestrians'])))
                                                                    ) for x in range(0,12)]
        stats['Overall']['Total Number of Persons Killed in Year'] = death_persons_total
        stats['Overall']['Total Number of Persons Killed Per Month'] = [str(
                                                                    (months(x+1), sum(map(lambda a: a[0]*a[1], md_arr[x][2]['Cyclist'])) + 
                                                                                  sum(map(lambda a: a[0]*a[1], md_arr[x][2]['Motorist'])) + 
                                                                                  sum(map(lambda a: a[0]*a[1], md_arr[x][2]['Pedestrians'])))
                                                                    ) for x in range(0,12)]
        stats['Overall']['Total Number of Collisions in Year'] = total
        stats['Overall']['Total Number of Collisions w/ Injury in Year'] = injury_total
        stats['Overall']['Total Number of Collisions w/ Death in Year'] = death_total
        stats['Overall']['Average Collisions Per Month'] = total // 12
        stats['Overall']['Average Collisions Per Day'] = total // 365
        stats['Overall']['Monthly Adjusted Average Collision Weight (%)'] = [str(
                                                                    (months(x+1), round((float(m_arr[x][1])/float(total)*100.0),2))
                                                                    ) for x in range(0,12)]
        stats['Overall']['Collisions Per Month'] = [str(
                                                    (months(x+1), m_arr[x][1])
                                                    ) for x in range(0,12)]
        stats['Overall']['Average Collisions Per Day (Monthly)'] = [str(
                                                                    (months(x+1), m_arr[x][1]/monthrange(int(str(20)+str(yr)), x+1)[1])
                                                                    ) for x in range(0,12)]
        stats['Overall']['Injury Rate (Year %)'] = float(injury_total)/float(total) * 100.00
        stats['Overall']['Death Rate (Year %)'] = float(death_total)/float(total) * 100.00
        stats['Overall']['Number of Collisions w/ Injuries Per Month'] = [str(
                                                                        (months(x+1), mi_arr[x][1])
                                                                        ) for x in range(0,12)]
        stats['Overall']['Number of Collisions w/ Deaths Per Month'] = [str(
                                                                        (months(x+1), md_arr[x][1])
                                                                        ) for x in range(0,12)]
        stats['Overall']['Injury Collision Rate Per Month (Month %)'] = [str(
                                                            (months(x+1), round((float(mi_arr[x][1])/float(m_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if m_arr[x][1] != 0]
        stats['Overall']['Death Collision Rate Per Month (Month %)'] = [str(
                                                            (months(x+1), round((float(md_arr[x][1])/float(m_arr[x][1]) * 100.00),2))
                                                            ) for x in range(0,12) if m_arr[x][1] != 0]






        # Rain Statistics
        stats['Rain'] = {}
        r_arr = [] # Rain Collision Data
        ri_arr = [] # Rain Collision Injury Data
        rd_arr = [] # Rain Collision Death Data

        """PREPROCESSING"""
        # Rain Collision Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Rain="1" and DATETIME like "{0}"'.format(date)
            sq = sqlContext.sql(query)
            r_arr.append((x, sq.count()))
            sq.unpersist()
        rain_total = sum([x[1] for x in r_arr])

        # Rain Collision Injury Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Rain="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                    NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
            df_injs = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
            ri_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_injs.unpersist()
        rain_injury_total = sum([x[1] for x in ri_arr])
        rain_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in ri_arr])

        # Rain Collision Death Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Rain="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                    NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
            df_deths = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
            rd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_deths.unpersist()

        rain_death_total = sum([x[1] for x in rd_arr])
        rain_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in rd_arr])




        # Statistics Deriving
        stats['Rain']['Total Number of Persons Injured in Collision w/ Rain'] = rain_injury_persons_total
        stats['Rain']['Total Number of Persons Killed in Collision w/ Rain'] = rain_death_persons_total
        stats['Rain']['Total Number of Persons Injured in Collision w/ Rain Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], ri_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], ri_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], ri_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['Rain']['Total Number of Persons Killed in Collision w/ Rain Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], rd_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], rd_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], rd_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['Rain']['Total Number of Collisions w/ Injury & Rain in Year'] = rain_injury_total
        stats['Rain']['Total Number of Collisions w/ Death & Rain in Year'] = rain_death_total
        stats['Rain']['Injury Rate w/ Rain (Year %)'] = float(rain_injury_total)/float(rain_total) * 100.00
        stats['Rain']['Death Rate w/ Rain (Year %)'] = float(rain_death_total)/float(rain_total) * 100.00
        stats['Rain']['Total Number of Rain Collisions in Year'] = rain_total
        stats['Rain']['Percent of Collisions in Rain Over Year (%)'] = (float(rain_total) / float(total)) * 100.00
        stats['Rain']['Collisions in Rain'] = [str(
                                            (months(x+1), r_arr[x][1])
                                            ) for x in range(0,12)]
        stats['Rain']['Average Collisions in Rain Per Month'] = rain_total//12
        stats['Rain']['Number of Collisions w/ Injuries and Rain Per Month'] = [str(
                                                                                (months(x+1), ri_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['Rain']['Number of Collisions w/ Deaths and Rain Per Month'] = [str(
                                                                                (months(x+1), rd_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['Rain']['Percent of Collisions in Each Month That Are Rain Collisions (Rain %)'] = [str(
                                                                (months(x+1), round((float(r_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                                ) for x in range(0,12) if m_arr[x][1] != 0]
        stats['Rain']['Injury Rate Per Month w/ Rain (Month %)'] = [str(
                                                                (months(x+1), round((float(ri_arr[x][1])/float(r_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if r_arr[x][1] != 0]
        stats['Rain']['Death Rate Per Month w/ Rain (Month %)'] = [str(
                                                                (months(x+1), round((float(rd_arr[x][1])/float(r_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if r_arr[x][1] != 0]




        # Snow Statistics
        stats['Snow'] = {}
        s_arr = [] # Snow Collision Data
        si_arr = [] # Snow Collision Injury Data
        sd_arr = [] # Snow Collision Death Data

        """PREPROCESSING"""
        # Snow Collision Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Snow="1" and DATETIME like "{0}"'.format(date)
            sq = sqlContext.sql(query)
            s_arr.append((x, sq.count()))
            sq.unpersist()
        snow_total = sum([x[1] for x in s_arr])

        # Snow Collision Injury Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Snow="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                    NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
            df_injs = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
            si_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_injs.unpersist()
        snow_injury_total = sum([x[1] for x in si_arr])
        snow_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in si_arr])

        # Snow Collision Death Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Snow="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                    NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
            df_deths = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
            sd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_deths.unpersist()

        snow_death_total = sum([x[1] for x in sd_arr])
        snow_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in sd_arr])




        # Statistics Deriving
        stats['Snow']['Total Number of Persons Injured in Collision w/ Snow'] = snow_injury_persons_total
        stats['Snow']['Total Number of Persons Killed in Collision w/ Snow'] = snow_death_persons_total
        stats['Snow']['Total Number of Persons Injured in Collision w/ Snow Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], si_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], si_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], si_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['Snow']['Total Number of Persons Killed in Collision w/ Snow Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], sd_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], sd_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], sd_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['Snow']['Total Number of Collisions w/ Injury & Snow in Year'] = snow_injury_total
        stats['Snow']['Total Number of Collisions w/ Death & Snow in Year'] = snow_death_total
        stats['Snow']['Injury Rate w/ Snow (Year %)'] = float(snow_injury_total)/float(snow_total) * 100.00
        stats['Snow']['Death Rate w/ Snow (Year %)'] = float(snow_death_total)/float(snow_total) * 100.00
        stats['Snow']['Total Number of Snow Collisions in Year'] = snow_total
        stats['Snow']['Percent of Collisions in Snow Over Year (%)'] = (float(snow_total) / float(total)) * 100.00
        stats['Snow']['Collisions in Snow'] = [str(
                                            (months(x+1), s_arr[x][1])
                                            ) for x in range(0,12)]
        stats['Snow']['Average Collisions in Snow Per Month'] = snow_total//12
        stats['Snow']['Number of Collisions w/ Injuries and Snow Per Month'] = [str(
                                                                                (months(x+1), si_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['Snow']['Number of Collisions w/ Deaths and Snow Per Month'] = [str(
                                                                                (months(x+1), sd_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['Snow']['Percent of Collisions in Each Month That Are Snow Collisions (Snow %)'] = [str(
                                                                (months(x+1), round((float(s_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                                ) for x in range(0,12) if m_arr[x][1] != 0]
        stats['Snow']['Injury Rate Per Month w/ Snow (Month %)'] = [str(
                                                                (months(x+1), round((float(si_arr[x][1])/float(s_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if s_arr[x][1] != 0]
        stats['Snow']['Death Rate Per Month w/ Snow (Month %)'] = [str(
                                                                (months(x+1), round((float(sd_arr[x][1])/float(s_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if s_arr[x][1] != 0]








        # Freezing Temperatures Statistics
        stats['Freezing'] = {}
        f_arr = [] # Freezing Collision Data
        fi_arr = [] # Freezing Collision Injury Data
        fd_arr = [] # Freezing Collision Death Data

        """PREPROCESSING"""
        # Freezing Collision Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Freezing="1" and DATETIME like "{0}"'.format(date)
            sq = sqlContext.sql(query)
            f_arr.append((x, sq.count()))
            sq.unpersist()
        freezing_total = sum([x[1] for x in f_arr])

        # Freezing Collision Injury Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Freezing="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                    NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
            df_injs = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
            fi_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_injs.unpersist()
        freezing_injury_total = sum([x[1] for x in fi_arr])
        freezing_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in fi_arr])

        # Freezing Collision Death Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Freezing="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                    NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
            df_deths = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
            fd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_deths.unpersist()

        freezing_death_total = sum([x[1] for x in fd_arr])
        freezing_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in fd_arr])




        # Statistics Deriving
        stats['Freezing']['Total Number of Persons Injured in Collision w/ Freezing'] = freezing_injury_persons_total
        stats['Freezing']['Total Number of Persons Killed in Collision w/ Freezing'] = freezing_death_persons_total
        stats['Freezing']['Total Number of Persons Injured in Collision w/ Freezing Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], fi_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], fi_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], fi_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['Freezing']['Total Number of Persons Killed in Collision w/ Freezing Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], fd_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], fd_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], fd_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['Freezing']['Total Number of Collisions w/ Injury & Freezing in Year'] = freezing_injury_total
        stats['Freezing']['Total Number of Collisions w/ Death & Freezing in Year'] = freezing_death_total
        stats['Freezing']['Injury Rate w/ Freezing (Year %)'] = float(freezing_injury_total)/float(freezing_total) * 100.00
        stats['Freezing']['Death Rate w/ Freezing (Year %)'] = float(freezing_death_total)/float(freezing_total) * 100.00
        stats['Freezing']['Total Number of Freezing Collisions in Year'] = freezing_total
        stats['Freezing']['Percent of Collisions in Freezing Over Year (%)'] = (float(freezing_total) / float(total)) * 100.00
        stats['Freezing']['Collisions in Freezing'] = [str(
                                            (months(x+1), f_arr[x][1])
                                            ) for x in range(0,12)]
        stats['Freezing']['Average Collisions in Freezing Per Month'] = freezing_total//12
        stats['Freezing']['Number of Collisions w/ Injuries and Freezing Per Month'] = [str(
                                                                                (months(x+1), fi_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['Freezing']['Number of Collisions w/ Deaths and Freezing Per Month'] = [str(
                                                                                (months(x+1), fd_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['Freezing']['Percent of Collisions in Each Month That Are Freezing Collisions (Freezing %)'] = [str(
                                                                (months(x+1), round((float(f_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                                ) for x in range(0,12) if m_arr[x][1] != 0]
        stats['Freezing']['Injury Rate Per Month w/ Freezing (Month %)'] = [str(
                                                                (months(x+1), round((float(fi_arr[x][1])/float(f_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if f_arr[x][1] != 0]
        stats['Freezing']['Death Rate Per Month w/ Freezing (Month %)'] = [str(
                                                                (months(x+1), round((float(fd_arr[x][1])/float(f_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if f_arr[x][1] != 0]





        # Low Visibility Statistics
        stats['LowVisibility'] = {}
        lv_arr = [] # Low Visibility Collision Data
        lvi_arr = [] # Low Visibility Collision Injury Data
        lvd_arr = [] # Low Visibility Collision Death Data

        """PREPROCESSING"""
        # Low Visibility Collision Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where LowVisibility="1" and DATETIME like "{0}"'.format(date)
            sq = sqlContext.sql(query)
            lv_arr.append((x, sq.count()))
            sq.unpersist()
        lowvisibility_total = sum([x[1] for x in lv_arr])

        # Low Visibility Collision Injury Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where LowVisibility="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                    NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
            df_injs = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
            lvi_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_injs.unpersist()
        lowvisibility_injury_total = sum([x[1] for x in lvi_arr])
        lowvisibility_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in lvi_arr])

        # Low Visibility Collision Death Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where LowVisibility="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                    NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
            df_deths = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
            lvd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_deths.unpersist()

        lowvisibility_death_total = sum([x[1] for x in lvd_arr])
        lowvisibility_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in lvd_arr])




        # Statistics Deriving
        stats['LowVisibility']['Total Number of Persons Injured in Collision w/ Low Visibility'] = lowvisibility_injury_persons_total
        stats['LowVisibility']['Total Number of Persons Killed in Collision w/ Low Visibility'] = lowvisibility_death_persons_total
        stats['LowVisibility']['Total Number of Persons Injured in Collision w/ Low Visibility Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], lvi_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], lvi_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], lvi_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['LowVisibility']['Total Number of Persons Killed in Collision w/ Low Visibility Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], lvd_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], lvd_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], lvd_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['LowVisibility']['Total Number of Collisions w/ Injury & Low Visibility in Year'] = lowvisibility_injury_total
        stats['LowVisibility']['Total Number of Collisions w/ Death & Low Visibility in Year'] = lowvisibility_death_total
        stats['LowVisibility']['Injury Rate w/ Low Visibility (Year %)'] = float(lowvisibility_injury_total)/float(lowvisibility_total) * 100.00
        stats['LowVisibility']['Death Rate w/ Low Visibility (Year %)'] = float(lowvisibility_death_total)/float(lowvisibility_total) * 100.00
        stats['LowVisibility']['Total Number of Low Visibility Collisions in Year'] = lowvisibility_total
        stats['LowVisibility']['Percent of Collisions in Low Visibility Over Year (%)'] = (float(lowvisibility_total) / float(total)) * 100.00
        stats['LowVisibility']['Collisions in Low Visibility'] = [str(
                                            (months(x+1), lv_arr[x][1])
                                            ) for x in range(0,12)]
        stats['LowVisibility']['Average Collisions in Low Visibility Per Month'] = lowvisibility_total//12
        stats['LowVisibility']['Number of Collisions w/ Injuries and Low Visibility Per Month'] = [str(
                                                                                (months(x+1), lvi_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['LowVisibility']['Number of Collisions w/ Deaths and Low Visibility Per Month'] = [str(
                                                                                (months(x+1), lvd_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['LowVisibility']['Percent of Collisions in Each Month That Are Low Visibility Collisions (LowVisibility %)'] = [str(
                                                                (months(x+1), round((float(lv_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                                ) for x in range(0,12) if m_arr[x][1] != 0]
        stats['LowVisibility']['Injury Rate Per Month w/ Low Visibility (Month %)'] = [str(
                                                                (months(x+1), round((float(lvi_arr[x][1])/float(lv_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if lv_arr[x][1] != 0]
        stats['LowVisibility']['Death Rate Per Month w/ Low Visibility (Month %)'] = [str(
                                                                (months(x+1), round((float(lvd_arr[x][1])/float(lv_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if lv_arr[x][1] != 0]    





        # Windy Statistics
        stats['Windy'] = {}
        w_arr = [] # Windy Collision Data
        wi_arr = [] # Windy Collision Injury Data
        wd_arr = [] # Windy Collision Death Data

        """PREPROCESSING"""
        # Windy Collision Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Windy="1" and DATETIME like "{0}"'.format(date)
            sq = sqlContext.sql(query)
            w_arr.append((x, sq.count()))
            sq.unpersist()
        windy_total = sum([x[1] for x in w_arr])

        # Windy Collision Injury Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Windy="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_INJURED!=0 or \
                                                                    NUMBER_OF_MOTORIST_INJURED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_INJURED!=0)'.format(date)
            df_injs = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_CYCLIST_INJURED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_MOTORIST_INJURED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_injs.groupBy("NUMBER_OF_PEDESTRIANS_INJURED").count().orderBy('count', ascending=False).collect())
            wi_arr.append((x, df_injs.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_injs.unpersist()
        windy_injury_total = sum([x[1] for x in wi_arr])
        windy_injury_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in wi_arr])

        # Windy Collision Death Data Query
        for x in range(1,13):
            date = "20{1}-0{0}%".format(x, yr) if x < 10 else "20{1}-{0}%".format(x, yr)
            query = 'select * from df where Windy="1" and DATETIME like "{0}" and ( \
                                                                    NUMBER_OF_CYCLIST_KILLED!=0 or \
                                                                    NUMBER_OF_MOTORIST_KILLED!=0 or \
                                                                    NUMBER_OF_PEDESTRIANS_KILLED!=0)'.format(date)
            df_deths = sqlContext.sql(query)
            cyclist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_CYCLIST_KILLED").count().orderBy('count', ascending=False).collect())
            motorist = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_MOTORIST_KILLED").count().orderBy('count', ascending=False).collect())
            pedestrian = map(lambda x: (int(x[0]), x[1]), df_deths.groupBy("NUMBER_OF_PEDESTRIANS_KILLED").count().orderBy('count', ascending=False).collect())
            wd_arr.append((x, df_deths.count(), {"Cyclist": cyclist, "Motorist": motorist, "Pedestrians": pedestrian}))
            df_deths.unpersist()

        windy_death_total = sum([x[1] for x in wd_arr])
        windy_death_persons_total = sum([sum(map(lambda a: a[0]*a[1], x[2]['Cyclist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Motorist'])) + sum(map(lambda a: a[0]*a[1], x[2]['Pedestrians'])) for x in wd_arr])




        # Statistics Deriving
        stats['Windy']['Total Number of Persons Injured in Collision w/ Windy'] = windy_injury_persons_total
        stats['Windy']['Total Number of Persons Killed in Collision w/ Windy'] = windy_death_persons_total
        stats['Windy']['Total Number of Persons Injured in Collision w/ Windy Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], wi_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], wi_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], wi_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['Windy']['Total Number of Persons Killed in Collision w/ Windy Per Month'] = [str(
                                                                            (months(x+1), sum(map(lambda a: a[0]*a[1], wd_arr[x][2]['Cyclist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], wd_arr[x][2]['Motorist'])) + 
                                                                                          sum(map(lambda a: a[0]*a[1], wd_arr[x][2]['Pedestrians'])))
                                                                            ) for x in range(0,12)]    
        stats['Windy']['Total Number of Collisions w/ Injury & Windy in Year'] = windy_injury_total
        stats['Windy']['Total Number of Collisions w/ Death & Windy in Year'] = windy_death_total
        stats['Windy']['Injury Rate w/ Windy (Year %)'] = float(windy_injury_total)/float(windy_total) * 100.00
        stats['Windy']['Death Rate w/ Windy (Year %)'] = float(windy_death_total)/float(windy_total) * 100.00
        stats['Windy']['Total Number of Windy Collisions in Year'] = windy_total
        stats['Windy']['Percent of Collisions in Windy Over Year (%)'] = (float(windy_total) / float(total)) * 100.00
        stats['Windy']['Collisions in Windy'] = [str(
                                            (months(x+1), w_arr[x][1])
                                            ) for x in range(0,12)]
        stats['Windy']['Average Collisions in Windy Per Month'] = windy_total//12
        stats['Windy']['Number of Collisions w/ Injuries and Windy Per Month'] = [str(
                                                                                (months(x+1), wi_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['Windy']['Number of Collisions w/ Deaths and Windy Per Month'] = [str(
                                                                                (months(x+1), wd_arr[x][1])
                                                                                ) for x in range(0,12)]
        stats['Windy']['Percent of Collisions in Each Month That Are Windy Collisions (Windy %)'] = [str(
                                                                (months(x+1), round((float(w_arr[x][1])/float(m_arr[x][1]))*100.0, 2))
                                                                ) for x in range(0,12) if m_arr[x][1] != 0]
        stats['Windy']['Injury Rate Per Month w/ Windy (Month %)'] = [str(
                                                                (months(x+1), round((float(wi_arr[x][1])/float(w_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if w_arr[x][1] != 0]
        stats['Windy']['Death Rate Per Month w/ Windy (Month %)'] = [str(
                                                                (months(x+1), round((float(wd_arr[x][1])/float(w_arr[x][1]) * 100.00),2))
                                                                ) for x in range(0,12) if w_arr[x][1] != 0]









        json_results = json.dumps(stats, indent=4, sort_keys=True)
        #print json_results
        path = "hdfs:///user/dzeng01/project/statistical_results_20{0}".format(yr)
        sc.parallelize([json_results]).coalesce(1).saveAsTextFile(path)


        # Finished with year processing. Clear and move on.
        sqlContext.dropTempTable('df')
        df.unpersist()





ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at c:\python27\lib\site-packages\IPython\utils\py3compat.py:279 